## Homework 6
author : Hongliang Chi, hc962@cornell.edu

In [1]:
from gurobipy import *
import pandas as pd

### Question 1

In [2]:
path_data = pd.read_csv('shortest_path_data-1.txt', delimiter= ' ')
path_data.columns = ['destination','cost']
path_data['origin'] = path_data.index
path_data = path_data[['origin','destination','cost']].reset_index(drop =True)
path_data.origin = path_data.origin.astype('int')
path_data.destination = path_data.destination.astype('int')
path_data.cost = path_data.cost.astype('float')

In [3]:
path_data

,origin,destination,cost
0,1,2,1.0
1,1,3,2.0
2,2,3,1.0
3,2,4,5.0
4,2,5,2.0
5,3,4,2.0
6,3,5,1.0
7,3,6,4.0
8,4,5,3.0
9,4,6,6.0


In [4]:
myModel = Model( "shortest_path" )
start_node = 1 
end_node = 8 
index_list = [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination)]
coef_list =  [int(i) for i in path_data.cost]
var_dic = {}
constraint = {}

for x in index_list:
    var_dic[x] = myModel.addVar(vtype = GRB.CONTINUOUS , name = "x%s"%str(x))
myModel.update()

objExpr = LinExpr()
for coef, index in zip(coef_list,index_list):
    objExpr += coef * var_dic[index]
myModel.setObjective( objExpr , GRB.MINIMIZE )

#start node loop 
Const_start = LinExpr()
for edge in [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination) if i == start_node]:
    Const_start += var_dic[edge]
myModel.addConstr( lhs = Const_start , sense = GRB.EQUAL , \
                   rhs = 1 , name = "Const_start-" + str(start_node) )

#end node loop 
Const_end = LinExpr()
for edge in [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination) if j == end_node]:
    Const_end += -1 * var_dic[edge]
myModel.addConstr( lhs = Const_end , sense = GRB.EQUAL , \
                   rhs = -1 , name = "Const_end-" + str(end_node) )
#rest node loop
for node in list(set(path_data.origin.to_list() + path_data.destination.to_list()) -set([start_node, end_node])):
    constExpr = LinExpr()
    for outbound in [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination) if i == node]:
        constExpr += var_dic[outbound]
    for inbound in [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination) if j == node]:
        constExpr += -1* var_dic[inbound]
    myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL , rhs = 0 , \
                       name = "node-" + str( node ) )

myModel.update()
# write the model in a file to make sure it is constructed correctly
myModel.write( filename = "shortest_path.lp" )

# optimize the model
myModel.optimize()

# check the status of the model
curStatus = myModel.status
if curStatus in (GRB.Status.INF_OR_UNBD, GRB.Status.INFEASIBLE, \
              GRB.Status.UNBOUNDED):
    print( "Could not find the optimal solution" )
    exit(1)

# print optimal objective and optimal solution
print( "\nOptimal Objective: " + str( myModel.ObjVal ) )
print( "\nOptimal Solution:" )
myVars = myModel.getVars()
for curVar in myVars:
    print ( curVar.varName + " " + str( curVar.x ) )
    

Academic license - for non-commercial use only
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.02s
Presolved: 6 rows, 14 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9920000e+00   2.004000e+00   0.000000e+00      0s
       3    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  8.000000000e+00

Optimal Objective: 8.0

Optimal Solution:
x12 0.0
x13 1.0
x23 0.0
x24 0.0
x25 0.0
x34 0.0
x35 1.0
x36 0.0
x45 0.0
x46 0.0
x47 0.0
x56 1.0
x57 0.0
x67 0.0
x68 1.0
x78 0.0
